# Brain Stroke prediction- Stack Model (6 Models)

## Context:

A stroke is a medical condition in which poor blood flow to the brain causes cell death. There are two main types of stroke: ischemic, due to lack of blood flow, and hemorrhagic, due to bleeding. Both cause parts of the brain to stop functioning properly. Signs and symptoms of a stroke may include an inability to move or feel on one side of the body, problems understanding or speaking, dizziness, or loss of vision to one side. Signs and symptoms often appear soon after the stroke has occurred. If symptoms last less than one or two hours, the stroke is a transient ischemic attack (TIA), also called a mini-stroke. A hemorrhagic stroke may also be associated with a severe headache. The symptoms of a stroke can be permanent. Long-term complications may include pneumonia and loss of bladder control.

The main risk factor for stroke is high blood pressure. Other risk factors include high blood cholesterol, tobacco smoking, obesity, diabetes mellitus, a previous TIA, end-stage kidney disease, and atrial fibrillation. An ischemic stroke is typically caused by blockage of a blood vessel, though there are also less common causes. A hemorrhagic stroke is caused by either bleeding directly into the brain or into the space between the brain's membranes. Bleeding may occur due to a ruptured brain aneurysm. Diagnosis is typically based on a physical exam and supported by medical imaging such as a CT scan or MRI scan. A CT scan can rule out bleeding, but may not necessarily rule out ischemia, which early on typically does not show up on a CT scan. Other tests such as an electrocardiogram (ECG) and blood tests are done to determine risk factors and rule out other possible causes. Low blood sugar may cause similar symptoms.

Prevention includes decreasing risk factors, surgery to open up the arteries to the brain in those with problematic carotid narrowing, and warfarin in people with atrial fibrillation. Aspirin or statins may be recommended by physicians for prevention. A stroke or TIA often requires emergency care. An ischemic stroke, if detected within three to four and half hours, may be treatable with a medication that can break down the clot. Some hemorrhagic strokes benefit from surgery. Treatment to attempt recovery of lost function is called stroke rehabilitation, and ideally takes place in a stroke unit; however, these are not available in much of the world.

### Attribute Information:

1) gender: "Male", "Female" or "Other"
2) age: age of the patient
3) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
4) heartdisease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease 5) evermarried: "No" or "Yes"
6) worktype: "children", "Govtjov", "Neverworked", "Private" or "Self-employed" 7) Residencetype: "Rural" or "Urban"
8) avgglucoselevel: average glucose level in blood
9) bmi: body mass index
10) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
11) stroke: 1 if the patient had a stroke or 0 if not

*Note: "Unknown" in smoking_status means that the information is unavailable for this patient

## IMPORTING LIBRARIES AND LOADING DATA

In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

import joblib as joblib

In [2]:
df = pd.read_csv('trainFile.csv')
df.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0
5,32257,Female,47.0,0,0,Yes,Private,Urban,210.95,50.1,NaN,0
6,52800,Female,52.0,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
7,41413,Female,75.0,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
8,15266,Female,32.0,0,0,Yes,Private,Rural,77.67,32.3,smokes,0
9,28674,Female,74.0,1,0,Yes,Self-employed,Urban,205.84,54.6,never smoked,0


## DATA EXPLORATION

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43400 entries, 0 to 43399
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 43400 non-null  int64  
 1   gender             43400 non-null  object 
 2   age                43400 non-null  float64
 3   hypertension       43400 non-null  int64  
 4   heart_disease      43400 non-null  int64  
 5   ever_married       43400 non-null  object 
 6   work_type          43400 non-null  object 
 7   Residence_type     43400 non-null  object 
 8   avg_glucose_level  43400 non-null  float64
 9   bmi                41938 non-null  float64
 10  smoking_status     30108 non-null  object 
 11  stroke             43400 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 4.0+ MB


In [4]:
df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,43400.000000,43400.000000,43400.000000,43400.000000,43400.000000,41938.000000,43400.000000
mean,36326.142350,42.217894,0.093571,0.047512,104.482750,28.605038,0.018041
std,21072.134879,22.519649,0.291235,0.212733,43.111751,7.770020,0.133103
min,1.000000,0.080000,0.000000,0.000000,55.000000,10.100000,0.000000
25%,18038.500000,24.000000,0.000000,0.000000,77.540000,23.200000,0.000000
50%,36351.500000,44.000000,0.000000,0.000000,91.580000,27.700000,0.000000
75%,54514.250000,60.000000,0.000000,0.000000,112.070000,32.900000,0.000000
max,72943.000000,82.000000,1.000000,1.000000,291.050000,97.600000,1.000000


In [5]:
print(df['gender'].unique())
print(df['work_type'].unique())
print(df['Residence_type'].unique())
print(df['smoking_status'].unique())
print(df['ever_married'].unique())

['Male' 'Female' 'Other']
['children' 'Private' 'Never_worked' 'Self-employed' 'Govt_job']
['Rural' 'Urban']
[nan 'never smoked' 'formerly smoked' 'smokes']
['No' 'Yes']


## DATA PREPROCESSING

In [6]:
df['ever_married'] = [ 0 if i !='Yes' else 1 for i in df['ever_married'] ]
df['gender'] = [0 if i != 'Female' else 1 for i in df['gender']]
df.head(5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,0,3.0,0,0,0,children,Rural,95.12,18.0,NaN,0
1,30468,0,58.0,1,0,1,Private,Urban,87.96,39.2,never smoked,0
2,16523,1,8.0,0,0,0,Private,Urban,110.89,17.6,NaN,0
3,56543,1,70.0,0,0,1,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,0,14.0,0,0,0,Never_worked,Rural,161.28,19.1,NaN,0


In [7]:
df = pd.get_dummies(df, columns = ['work_type', 'Residence_type','smoking_status'])
df.sample(5)

,id,gender,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
34962,27761,1,4.0,0,0,0,84.63,19.2,0,0,0,0,0,1,1,0,0,0,0
17853,69400,1,30.0,0,0,1,64.75,26.9,0,0,0,1,0,0,0,1,0,0,0
13411,58299,1,49.0,0,0,1,83.13,23.1,0,1,0,0,0,0,0,1,1,0,0
28522,27222,1,46.0,0,0,1,103.01,38.8,0,0,0,1,0,0,1,0,0,0,1
12142,58870,0,38.0,0,0,0,106.39,30.6,0,0,0,0,1,0,1,0,0,0,1


In [8]:
df.isnull().sum()

id                                   0
gender                               0
age                                  0
hypertension                         0
heart_disease                        0
ever_married                         0
avg_glucose_level                    0
bmi                               1462
stroke                               0
work_type_Govt_job                   0
work_type_Never_worked               0
work_type_Private                    0
work_type_Self-employed              0
work_type_children                   0
Residence_type_Rural                 0
Residence_type_Urban                 0
smoking_status_formerly smoked       0
smoking_status_never smoked          0
smoking_status_smokes                0
dtype: int64

In [9]:
df = df.dropna(how = 'any', axis=0)

In [10]:
df.to_excel('datasetCleaned.xlsx')

In [11]:
X = df.drop(['stroke','id'], axis = 1)
y = df['stroke']

In [12]:
X.head(10)

,gender,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0,3.0,0,0,0,95.12,18.0,0,0,0,0,1,1,0,0,0,0
1,0,58.0,1,0,1,87.96,39.2,0,0,1,0,0,0,1,0,1,0
2,1,8.0,0,0,0,110.89,17.6,0,0,1,0,0,0,1,0,0,0
3,1,70.0,0,0,1,69.04,35.9,0,0,1,0,0,1,0,1,0,0
4,0,14.0,0,0,0,161.28,19.1,0,1,0,0,0,1,0,0,0,0
5,1,47.0,0,0,1,210.95,50.1,0,0,1,0,0,0,1,0,0,0
6,1,52.0,0,0,1,77.59,17.7,0,0,1,0,0,0,1,1,0,0
7,1,75.0,0,1,1,243.53,27.0,0,0,0,1,0,1,0,0,1,0
8,1,32.0,0,0,1,77.67,32.3,0,0,1,0,0,1,0,0,0,1
9,1,74.0,1,0,1,205.84,54.6,0,0,0,1,0,0,1,0,1,0


### Target and Feature values / Train Test Split

In [13]:
X_train, X_test, y_train , y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((33550, 17), (8388, 17))

## MODEL BUILDING

### KNeighborsClassifier

Classifier implementing the k-nearest neighbors vote.

class sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)

In [14]:
knn = KNeighborsClassifier(3) # Define classifier
knn.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

# Training set performance
knn_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
knn_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
knn_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
knn_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
knn_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
knn_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % knn_train_accuracy)
print('- MCC: %s' % knn_train_mcc)
print('- F1 score: %s' % knn_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % knn_test_accuracy)
print('- MCC: %s' % knn_test_mcc)
print('- F1 score: %s' % knn_test_f1)

Model performance for Training set
- Accuracy: 0.9855141579731743
- MCC: 0.2714217088023517
- F1 score: 0.9803223813632442
----------------------------------
Model performance for Test set
- Accuracy: 0.9834287076776347
- MCC: -0.004865438031361363
- F1 score: 0.9767491714651221


### Support Vector Classification (SVC)

C-Support Vector Classification.
SVC, NuSVC and LinearSVC are classes capable of performing binary and multi-class classification on a dataset.

The implementation is based on libsvm. The fit time scales at least quadratically with the number of samples and may be impractical beyond tens of thousands of samples.

class sklearn.svm.SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)

In [15]:
svm_rbf = SVC(gamma='auto', C=1)
svm_rbf.fit(X_train, y_train)

# Make predictions
y_train_pred = svm_rbf.predict(X_train)
y_test_pred = svm_rbf.predict(X_test)

# Training set performance
svm_rbf_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
svm_rbf_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
svm_rbf_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
svm_rbf_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
svm_rbf_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
svm_rbf_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % svm_rbf_train_accuracy)
print('- MCC: %s' % svm_rbf_train_mcc)
print('- F1 score: %s' % svm_rbf_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % svm_rbf_test_accuracy)
print('- MCC: %s' % svm_rbf_test_mcc)
print('- F1 score: %s' % svm_rbf_test_f1)

Model performance for Training set
- Accuracy: 0.9845901639344262
- MCC: 0.0
- F1 score: 0.9769450726235194
----------------------------------
Model performance for Test set
- Accuracy: 0.9849785407725322
- MCC: 0.0
- F1 score: 0.9775246491126318


### Decision Tree Classifier

A decision tree classifier.
Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.

class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)

In [16]:
dt = DecisionTreeClassifier(max_depth=5) # Define classifier
dt.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

# Training set performance
dt_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
dt_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
dt_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
dt_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
dt_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
dt_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % dt_train_accuracy)
print('- MCC: %s' % dt_train_mcc)
print('- F1 score: %s' % dt_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % dt_test_accuracy)
print('- MCC: %s' % dt_test_mcc)
print('- F1 score: %s' % dt_test_f1)

Model performance for Training set
- Accuracy: 0.9848286140089418
- MCC: 0.12344663420900479
- F1 score: 0.9775321005492623
----------------------------------
Model performance for Test set
- Accuracy: 0.9845016690510253
- MCC: -0.002697410465010962
- F1 score: 0.9772861696142702


### Random Forest Classifier

A random forest classifier.

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree.

class sklearn.ensemble.RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

In [17]:
rf = RandomForestClassifier(n_estimators=10) # Define classifier
rf.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

# Training set performance
rf_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
rf_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
rf_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
rf_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
rf_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
rf_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % rf_train_accuracy)
print('- MCC: %s' % rf_train_mcc)
print('- F1 score: %s' % rf_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % rf_test_accuracy)
print('- MCC: %s' % rf_test_mcc)
print('- F1 score: %s' % rf_test_f1)

Model performance for Training set
- Accuracy: 0.9966318926974664
- MCC: 0.8824779377317886
- F1 score: 0.9964281470707034
----------------------------------
Model performance for Test set
- Accuracy: 0.9843824511206486
- MCC: -0.003015976451863439
- F1 score: 0.9772265318304354


### Multi-layer Perceptron classifier

This model optimizes the log-loss function using LBFGS or stochastic gradient descent.

class sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)

In [18]:
mlp = MLPClassifier(alpha=1, max_iter=1000)
mlp.fit(X_train, y_train)

# Make predictions
y_train_pred = mlp.predict(X_train)
y_test_pred = mlp.predict(X_test)

# Training set performance
mlp_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
mlp_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
mlp_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
mlp_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
mlp_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
mlp_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % mlp_train_accuracy)
print('- MCC: %s' % mlp_train_mcc)
print('- F1 score: %s' % mlp_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % mlp_test_accuracy)
print('- MCC: %s' % mlp_test_mcc)
print('- F1 score: %s' % mlp_test_f1)

Model performance for Training set
- Accuracy: 0.9845901639344262
- MCC: 0.0
- F1 score: 0.9769450726235194
----------------------------------
Model performance for Test set
- Accuracy: 0.9849785407725322
- MCC: 0.0
- F1 score: 0.9775246491126318


### Stacking Classifier

Stack of estimators with a final classifier.

Stacked generalization consists in stacking the output of individual estimator and use a classifier to compute the final prediction. Stacking allows to use the strength of each individual estimator by using their output as input of a final estimator.

Note that estimators_ are fitted on the full X while final_estimator_ is trained using cross-validated predictions of the base estimators using cross_val_predict.

class sklearn.ensemble.StackingClassifier(estimators, final_estimator=None, *, cv=None, stack_method='auto', n_jobs=None, passthrough=False, verbose=0)

In [19]:
estimator_list = [
    ('knn',knn),
    ('svm_rbf',svm_rbf),
    ('dt',dt),
    ('rf',rf),
    ('mlp',mlp) ]

# Build stack model
stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=LogisticRegression()
)

# Train stacked model
stack_model.fit(X_train, y_train)

# Make predictions
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)

# Training set model performance
stack_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
stack_model_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
stack_model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set model performance
stack_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
stack_model_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
stack_model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('- MCC: %s' % stack_model_train_mcc)
print('- F1 score: %s' % stack_model_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)
print('- MCC: %s' % stack_model_test_mcc)
print('- F1 score: %s' % stack_model_test_f1)

Model performance for Training set
- Accuracy: 0.9849478390461996
- MCC: 0.15119964794841836
- F1 score: 0.977820298441443
----------------------------------
Model performance for Test set
- Accuracy: 0.9845016690510253
- MCC: -0.002697410465010962
- F1 score: 0.9772861696142702


In [23]:
# Run the Line 2 only then model changed!!!
# joblib.dump(stack_model, 'stroke-prediction-model.joblib')

model = joblib.load('stroke-prediction-model.joblib') # Model File

## PREDICTION TESTING

In [24]:
import random

count=0
array=[]
predictionOutcome=[0]

print("######################################################### PROGRAM STARTED #########################################################")

while predictionOutcome == [0]:
    arr1=np.random.randint(2, size=1) #Gender
    arr2=np.random.randint(83, size=1) #Age
    arr3=np.random.randint(2, size=1) #Hypertention
    arr4=np.random.randint(2, size=1) #Heart Disease
    arr5=np.random.randint(2, size=1) #Ever Married
    arr6=np.random.randint(272, size=1) #Avg Glucose Level
    arr7=np.random.randint(100, size=1) #bmi
    
    worktype= random.randint(0,4) #Work Type
    
    arr8=[0] #Govt Job
    arr9=[0] #Private
    arr10=[0] #Self-Employed
    arr11=[0] #Children
    
    if worktype == 0:
        arr8=[1] #Govt Job
        arr9=[0] #Private
        arr10=[0] #Self-Employed
        arr11=[0] #Children
    elif worktype == 1:
        arr8=[0] #Govt Job
        arr9=[1] #Private
        arr10=[0] #Self-Employed
        arr11=[0] #Children
    elif worktype == 2:
        arr8=[0] #Govt Job
        arr9=[0] #Private
        arr10=[1] #Self-Employed
        arr10=[0] #Children
    elif worktype == 3:
        arr8=[0] #Govt Job
        arr10=[0] #Self-Employed
        arr10=[0] #Self-Employed
        arr11=[1] #Children
    
    residencetype= random.randint(0,2) #Residence Type
    
    arr12=[0] #Rural
    arr17=[0] #Urban
    
    if worktype == 0:
        arr12=[1] #Rural
        arr17=[0] #Urban
    elif worktype == 1:
        arr12=[0] #Rural
        arr17=[1] #Urban
    
    smokingtype= random.randint(0,4) #Smoking Type
    
    arr13=[0] #Unknown
    arr14=[0] #Formerly Smoked
    arr15=[0] #Never Smoked
    arr16=[0] #Smokes
    
    if worktype == 0:
        arr13=[1] #Unknown
        arr14=[0] #Formerly Smoked
        arr15=[0] #Never Smoked
        arr16=[0] #Smokes
    elif worktype == 1:
        arr13=[0] #Unknown
        arr14=[1] #Formerly Smoked
        arr15=[0] #Never Smoked
        arr16=[0] #Smokes
    elif worktype == 2:
        arr13=[0] #Unknown
        arr14=[0] #Formerly Smoked
        arr15=[1] #Never Smoked
        arr16=[0] #Smokes
    elif worktype == 3:
        arr13=[0] #Unknown
        arr14=[0] #Formerly Smoked
        arr15=[0] #Never Smoked
        arr16=[1] #Smokes
    
    array = np.concatenate((arr1, arr2, arr3, arr4, arr5, arr6, arr7, arr8, arr9, arr10, arr11, arr12, arr17, arr13, arr14, arr15, arr16), axis=0)
    
    print("\nArray %d =" %count, array)
    
    predictionOutcome = model.predict([array])
    
    if predictionOutcome == 0:
        print("Model predicts NO STROKE = ", predictionOutcome)
        print("###################################################################################################################################")
    else:
        print("Model predicts STROKE = ", predictionOutcome)
    
    count+=1
else:
    print("######################################################### PROGRAM FINISHED #########################################################")


######################################################### PROGRAM STARTED #########################################################

Array 0 = [ 0 17  1  1  1 93  0  0  0  0  1  0  0  0  0  0  1]
Model predicts NO STROKE =  [0]
###################################################################################################################################

Array 1 = [ 1 12  0  1  1 65 36  0  0  0  0  0  0  0  0  0  0]
Model predicts NO STROKE =  [0]
###################################################################################################################################

Array 2 = [ 0 64  0  0  1 82 70  0  0  0  0  0  0  0  0  0  0]
Model predicts NO STROKE =  [0]
###################################################################################################################################

Array 3 = [  0   1   1   1   1 179  99   0   0   0   0   0   0   0   0   1   0]
Model predicts NO STROKE =  [0]
#######################################################################